In [1]:
import numpy as np
import sys
sys.path.append('../wofs_phi')
import config as c
import utilities
from sklearn.metrics import brier_score_loss as BS
import os
from shutil import copy
import time
import datetime as dt

In [2]:
class model_stats:
    
    def __init__(self, hazard, wofs_spinup_time, forecast_length, wofs_lead_time, train_radius, ver_radius, train_type, ver_type, model_type,
                 n_folds, use_avg_srs, dates):
        self.hazard = hazard
        self.wofs_spinup_time = wofs_spinup_time
        self.forecast_length = forecast_length
        self.train_radius = train_radius
        self.ver_radius = ver_radius
        self.lead_time = wofs_lead_time
        self.train_type = train_type
        self.ver_type = ver_type
        self.train_test_dir = '/work/ryan.martz/wofs_phi_data/%s_train/test_fcsts/%s' %(train_type, model_type)
        self.ver_test_dir = '/work/ryan.martz/wofs_phi_data/%s_train/test_fcsts/%s' %(ver_type, model_type)
        self.model_type = model_type
        self.n_folds = n_folds
        self.use_avg_srs = use_avg_srs
        self.dates = dates
    
    def get_all_test_sr_events_fname_dir(self, fold):
        start_min = self.lead_time
        end_min = self.lead_time + self.forecast_length
        train_save_dir = '%s/%s/wofslag_%s/length_%s/fold%s' %(self.train_test_dir, self.hazard, self.wofs_spinup_time, self.forecast_length, fold)
        ver_save_dir = '%s/%s/wofslag_%s/length_%s/fold%s' %(self.ver_test_dir, self.hazard, self.wofs_spinup_time, self.forecast_length, fold)
        if self.train_type == 'warnings':
            if self.use_avg_srs:
                all_srs_fname = '%s_%s_trained_all_rf_%s_avg_sr_probs_spinup%smin_length%smin_%s-%s_fold%s.npy' %(self.model_type, self.train_type, self.hazard,
                                                                                                           self.wofs_spinup_time, self.forecast_length,
                                                                                                           start_min, end_min, fold)
            else:
                all_srs_fname = '%s_%s_trained_all_rf_%s_sr_probs_spinup%smin_length%smin_%s-%s_fold%s.npy' %(self.model_type, self.train_type, self.hazard,
                                                                                                           self.wofs_spinup_time, self.forecast_length,
                                                                                                           start_min, end_min, fold)
            all_probs_fname = '%s_%s_trained_all_rf_%s_raw_probs_spinup%smin_length%smin_%s-%s_fold%s.npy' %(self.model_type, self.train_type, self.hazard,
                                                                                                       self.wofs_spinup_time, self.forecast_length,
                                                                                                       start_min, end_min, fold)
        else:
            if self.use_avg_srs:
                all_srs_fname = '%s_%s_trained_all_rf_%s_avg_sr_probs_spinup%smin_length%smin_%s-%s_r%skm_fold%s.npy' %(self.model_type, self.train_type, self.hazard,
                                                                                                                 self.wofs_spinup_time,
                                                                                                                 self.forecast_length, start_min, end_min,
                                                                                                                 self.train_radius, fold)
            else:
                all_srs_fname = '%s_%s_trained_all_rf_%s_sr_probs_spinup%smin_length%smin_%s-%s_r%skm_fold%s.npy' %(self.model_type, self.train_type, self.hazard,
                                                                                                                 self.wofs_spinup_time,
                                                                                                                 self.forecast_length, start_min, end_min,
                                                                                                                 self.train_radius, fold)
            all_probs_fname = '%s_%s_trained_all_rf_%s_raw_probs_spinup%smin_length%smin_%s-%s_r%skm_fold%s.npy' %(self.model_type, self.train_type, self.hazard,
                                                                                                       self.wofs_spinup_time, self.forecast_length,
                                                                                                       start_min, end_min, self.train_radius, fold)
        if self.ver_type == 'warnings':
            all_events_fname = '%s_%s_trained_all_%s_events_spinup%smin_length%smin_%s-%s_fold%s.npy' %(self.model_type, self.ver_type, self.hazard, self.wofs_spinup_time,
                                                                                                     self.forecast_length, start_min, end_min, fold)
        else:
            all_events_fname = '%s_%s_trained_all_%s_events_spinup%smin_length%smin_%s-%s_r%skm_fold%s.npy' %(self.model_type, self.ver_type, self.hazard,
                                                                                                           self.wofs_spinup_time, self.forecast_length,
                                                                                                           start_min, end_min, self.ver_radius, fold)
        
        return train_save_dir, ver_save_dir, all_srs_fname, all_probs_fname, all_events_fname
    
    def get_all_test_srs_probs_events_fold(self, fcst_dir, ver_dir, all_srs_fname, all_probs_fname, all_events_fname):
        all_srs_fold = np.load('%s/%s' %(fcst_dir, all_srs_fname))
        all_probs_fold = np.load('%s/%s' %(fcst_dir, all_probs_fname))
        all_events_fold = np.load('%s/%s' %(ver_dir, all_events_fname))
        return all_srs_fold, all_probs_fold, all_events_fold
    
    def get_all_test_srs_probs_events(self):
        all_srs = []
        all_probs = []
        all_events = []
        for fold in range(self.n_folds):
            all_fcst_dir, all_ver_dir, all_srs_fname, all_probs_fname, all_events_fname = self.get_all_test_sr_events_fname_dir(fold)
            print(all_srs_fname)
            print(all_probs_fname)
            print(all_events_fname)
            all_srs_fold, all_probs_fold, all_events_fold = self.get_all_test_srs_probs_events_fold(all_fcst_dir, all_ver_dir, all_srs_fname, all_probs_fname, all_events_fname)
            all_srs.extend(all_srs_fold)
            all_probs.extend(all_probs_fold)
            all_events.extend(all_events_fold)
        return np.array(all_srs), np.array(all_probs), np.array(all_events)
    
    def calc_reliability(self, bins):
        'bins should be given in decimal values'
        
        all_srs, all_probs, all_events = self.get_all_test_srs_probs_events()
        print(len(all_srs), len(all_probs), len(all_events))
        rel_sum_srs = 0
        rel_sum_probs = 0
        rel_climos_srs = []
        rel_climos_probs = []
        fcst_frequencies_srs = []
        fcst_frequencies_probs = []
        for i in range(len(bins)-1):
            bin_start = bins[i]
            bin_end = bins[i+1]
            events_srs = all_events[np.where((all_srs >= bin_start) & (all_srs < bin_end))]
            rel_srs = all_srs[np.where((all_srs >= bin_start) & (all_srs < bin_end))]
            events_probs = all_events[np.where((all_probs >= bin_start) & (all_probs < bin_end))]
            rel_probs = all_probs[np.where((all_probs >= bin_start) & (all_probs < bin_end))]
            
            if events_srs.size == 0:
                rel_climos_srs.append(-1)
            else:
                rel_climos_srs.append(np.mean(events_srs))
                rel_sum_srs += np.sum(np.power(rel_srs - events_srs, 2))
            
            if events_probs.size == 0:
                rel_climos_probs.append(-1)
            else:
                rel_climos_probs.append(np.mean(events_probs))
                rel_sum_probs += np.sum(np.power(rel_probs - events_probs, 2))
            
            fcst_frequencies_srs.append(rel_srs.size)
            fcst_frequencies_probs.append(rel_probs.size)
        
        N = all_srs.size
        rel_srs = rel_sum_srs/N
        rel_probs = rel_sum_probs/N
        
        ver_climo = np.mean(all_events)
        
        return fcst_frequencies_srs, fcst_frequencies_probs, rel_srs, rel_probs, rel_climos_srs, rel_climos_probs, ver_climo
    
    def set_chunk_splits(self):
        '''Does the k fold logic. The class stores a list of dates to train on, and this samples it into training, testing, validation for each fold.
        This allows us to just use the fold number as an index to each of the lists to get the list of dates for the current fold for testing, validation,
        and training.'''
        
        num_dates = len(self.dates)
        indices = np.arange(num_dates)
        splits = np.array_split(indices, self.n_folds)
        chunk_splits = []
        for i in range(len(splits)):
            split = splits[i]
            chunk_splits.append(split[0])
        chunk_splits.append(num_dates)
        self.date_test_folds = []
        self.date_val_folds = []
        self.date_train_folds = []
        if self.n_folds == 1:
            self.date_test_folds = [self.dates]
            self.date_val_folds = [self.dates]
            self.date_train_folds = [self.dates]
            return
        for i in range(self.n_folds):
            if i == 0:
                self.date_test_folds.append(self.dates[chunk_splits[i]:chunk_splits[i+1]])
                self.date_val_folds.append(self.dates[chunk_splits[-2]:chunk_splits[-1]])
                self.date_train_folds.append(self.dates[chunk_splits[i+1]:chunk_splits[i+1+(self.n_folds-2)]])
            elif i == self.n_folds - 1:
                self.date_test_folds.append(self.dates[chunk_splits[i]:chunk_splits[-1]])
                self.date_val_folds.append(self.dates[chunk_splits[i-1]:chunk_splits[i]])
                self.date_train_folds.append(self.dates[chunk_splits[0]:chunk_splits[self.n_folds-2]])
            elif (i+1+(self.n_folds-2)) > (self.n_folds):
                self.date_test_folds.append(self.dates[chunk_splits[i]:chunk_splits[i+1]])
                self.date_val_folds.append(self.dates[chunk_splits[i-1]:chunk_splits[i]])
                train_folds = self.dates[chunk_splits[i+1]:chunk_splits[-1]]
                train_folds.extend(self.dates[chunk_splits[0]:chunk_splits[i-1]])
                self.date_train_folds.append(train_folds)
            else:
                self.date_test_folds.append(self.dates[chunk_splits[i]:chunk_splits[i+1]])
                self.date_val_folds.append(self.dates[chunk_splits[i-1]:chunk_splits[i]])
                self.date_train_folds.append(self.dates[chunk_splits[i+1]:chunk_splits[i+1+(self.n_folds-2)]])
        
        return
            
    def calc_pod_sr(self, thresholds):
        all_srs, all_probs, all_events = self.get_all_test_srs_probs_events()
        pods_sr = []
        pods_probs = []
        srs_sr = []
        srs_probs = []
        thresholds_sr = []
        thresholds_probs = []
        for t in thresholds:
            yes_fcst_events_srs = all_events[np.where(all_srs >= t)]
            no_fcst_events_srs = all_events[np.where(all_srs < t)]
            
            a_srs = np.sum(yes_fcst_events_srs)
            b_srs = yes_fcst_events_srs.size - a_srs
            c_srs = np.sum(no_fcst_events_srs)
            d_srs = no_fcst_events_srs.size - c_srs
            
            if (a_srs + c_srs > 0) and (a_srs + b_srs > 0):
            
                pod_srs = a_srs/(a_srs+c_srs)
                sr_srs = a_srs/(a_srs+b_srs)

                thresholds_sr.append(t)
                srs_sr.append(sr_srs)
                pods_sr.append(pod_srs)
            
            else:
                thresholds_sr.append(-1)
                srs_sr.append(-1)
                pods_sr.append(-1)
            
            yes_fcst_events_probs = all_events[np.where(all_probs >= t)]
            no_fcst_events_probs = all_events[np.where(all_probs < t)]
            
            a_probs = np.sum(yes_fcst_events_probs)
            b_probs = yes_fcst_events_probs.size - a_probs
            c_probs = np.sum(no_fcst_events_probs)
            d_probs = no_fcst_events_probs.size - c_probs
            
            if (a_probs + c_probs > 0) and (a_probs + b_probs > 0):
                pod_probs = a_probs/(a_probs+c_probs)
                sr_probs = a_probs/(a_probs+b_probs)

                thresholds_probs.append(t)
                pods_probs.append(pod_probs)
                srs_probs.append(sr_probs)
            else:
                thresholds_probs.append(-1)
                pods_probs.append(-1)
                srs_probs.append(-1)
        
        return thresholds_sr, pods_sr, srs_sr, thresholds_probs, pods_probs, srs_probs
    
    def gen_brier_skill_scores(self):
        sr_skill_by_fold = []
        probs_skill_by_fold = []
        all_srs = []
        all_probs = []
        all_events = []
        for fold in range(self.n_folds):
            fcst_dir, ver_dir, all_srs_fname, all_probs_fname, all_events_fname = self.get_all_test_sr_events_fname_dir(fold)
            all_srs_fold, all_probs_fold, all_events_fold = self.get_all_test_srs_probs_events_fold(fcst_dir, ver_dir, all_srs_fname, all_probs_fname, all_events_fname)
            
            #probs_less_10_indices = np.where(all_probs_fold < 0.1)
            #all_probs_fold[probs_less_10_indices] = 0
            #all_srs_fold[probs_less_10_indices] = 0
            
            num_fcsts = all_srs_fold.size
            total_yes = np.sum(all_events_fold)
            climo = total_yes/num_fcsts
            climo_fcst_fold = np.ones(num_fcsts)*climo
            
            bs_sr_fold = BS(all_events_fold, all_srs_fold)
            bs_probs_fold = BS(all_events_fold, all_probs_fold)
            bs_climo_fold = BS(all_events_fold, climo_fcst_fold)
            
            bss_sr_fold = (bs_sr_fold - bs_climo_fold)/(-bs_climo_fold)
            bss_probs_fold = (bs_probs_fold - bs_climo_fold)/(-bs_climo_fold)
            
            sr_skill_by_fold.append(bss_sr_fold)
            probs_skill_by_fold.append(bss_probs_fold)
            
            all_srs.extend(all_srs_fold)
            all_probs.extend(all_probs_fold)
            all_events.extend(all_events_fold)
            
        num_fcsts = len(all_srs)
        total_yes = np.sum(all_events)
        climo = total_yes/num_fcsts
        climo_fcst = np.ones(num_fcsts)*climo
        
        bs_sr = BS(all_events, all_srs)
        bs_probs = BS(all_events, all_probs)
        bs_climo = BS(all_events, climo_fcst)
        
        bss_sr = (bs_sr - bs_climo)/(-bs_climo)
        bss_probs = (bs_probs - bs_climo)/(-bs_climo)
        
        return bss_sr, sr_skill_by_fold, bss_probs, probs_skill_by_fold

In [4]:
def get_bss_fname_dir(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = False):
    if train_type == 'warnings' and ver_type == 'warnings':
        save_dir = '%s/%s_trained/length_%s/%s' %(home_dir, train_type, forecast_length, hazard)
        if use_avg_srs:
            overall_sr_file = '%s_%s_trained_%s_verified_%s_%smin_bss_from_avg_sr_map_by_lead_time.npy' %(model_type, train_type, ver_type, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_trained_%s_verified_%s_%smin_bss_from_avg_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, ver_type, hazard, forecast_length)
        else:
            overall_sr_file = '%s_%s_trained_%s_verified_%s_%smin_bss_from_sr_map_by_lead_time.npy' %(model_type, train_type, ver_type, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_trained_%s_verified_%s_%smin_bss_from_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, ver_type, hazard, forecast_length)
        
        overall_probs_file = '%s_%s_trained_%s_verified_%s_%smin_bss_from_raw_probs_by_lead_time.npy' %(model_type, train_type, ver_type, hazard, forecast_length)
        probs_by_fold_file = '%s_%s_trained_%s_verified_%s_%smin_bss_from_raw_probs_by_lead_time_and_fold.npy' %(model_type, train_type, ver_type, hazard, forecast_length)
    
    elif train_type == 'warnings':
        save_dir = '%s/%s_trained/length_%s/%s' %(home_dir, train_type, forecast_length, hazard)
        if use_avg_srs:
            overall_sr_file = '%s_%s_trained_%s_%skm_verified_%s_%smin_bss_from_avg_sr_map_by_lead_time.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_trained_%s_%skm_verified_%s_%smin_%skm_bss_from_avg_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length)
        else:
            overall_sr_file = '%s_%s_trained_%s_%skm_verified_%s_%smin_bss_from_sr_map_by_lead_time.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_trained_%s_%skm_verified_%s_%smin_%skm_bss_from_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length)
        
        overall_probs_file = '%s_%s_trained_%s_%skm_verified_%s_%smin_%skm_bss_from_raw_probs_by_lead_time.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length)
        probs_by_fold_file = '%s_%s_trained_%s_%skm_verified_%s_%smin_%skm_bss_from_raw_probs_by_lead_time_and_fold.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length)
    
    elif ver_type == 'warnings':
        save_dir = '%s/%s_trained/length_%s/%s' %(home_dir, train_type, forecast_length, hazard)
        if use_avg_srs:
            overall_sr_file = '%s_%s_%skm_trained_%s_verified_%s_%smin_bss_from_avg_sr_map_by_lead_time.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_%skm_trained_%s_verified_%s_%smin_%skm_bss_from_avg_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
        else:
            overall_sr_file = '%s_%s_%skm_trained_%s_verified_%s_%smin_bss_from_sr_map_by_lead_time.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_%skm_trained_%s_verified_%s_%smin_%skm_bss_from_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
        
        overall_probs_file = '%s_%s_%skm_trained_%s_verified_%s_%smin_%skm_bss_from_raw_probs_by_lead_time.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
        probs_by_fold_file = '%s_%s_%skm_trained_%s_verified_%s_%smin_%skm_bss_from_raw_probs_by_lead_time_and_fold.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
    
    else:
        save_dir = '%s/%s_trained/length_%s/%s' %(home_dir, train_type, forecast_length, hazard)
        if use_avg_srs:
            overall_sr_file = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_bss_from_avg_sr_map_by_lead_time.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_bss_from_avg_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
        else:
            overall_sr_file = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_bss_from_sr_map_by_lead_time.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
            sr_by_fold_file = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_bss_from_sr_map_by_lead_time_and_fold.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
        
        overall_probs_file = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_bss_from_raw_probs_by_lead_time.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
        probs_by_fold_file = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_bss_from_raw_probs_by_lead_time_and_fold.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length)
    
    return save_dir, overall_sr_file, overall_probs_file, sr_by_fold_file, probs_by_fold_file
    

In [5]:
def get_bss_start_i(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = False):
    
    save_dir, overall_sr_file, overall_probs_file, sr_by_fold_file, probs_by_fold_file = get_bss_fname_dir(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = use_avg_srs)
    
    if not (os.path.isfile('%s/%s' %(save_dir, overall_sr_file)) and os.path.isfile('%s/%s' %(save_dir, overall_probs_file)) and os.path.isfile('%s/%s' %(save_dir, sr_by_fold_file)) and os.path.isfile('%s/%s' %(save_dir, probs_by_fold_file))):
        return 0
    
    npy = np.load('%s/%s' %(save_dir, overall_sr_file))
    return npy.size

In [6]:
def get_done_skills(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = False):
    
    save_dir, overall_sr_file, overall_probs_file, sr_by_fold_file, probs_by_fold_file = get_bss_fname_dir(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = use_avg_srs)
    
    overall_srs = np.load('%s/%s' %(save_dir, overall_sr_file))
    overall_probs = np.load('%s/%s' %(save_dir, overall_probs_file))
    sr_by_fold = np.load('%s/%s' %(save_dir, sr_by_fold_file))
    probs_by_fold = np.load('%s/%s' %(save_dir, probs_by_fold_file))
    
    return overall_srs, overall_probs, sr_by_fold, probs_by_fold


In [18]:
home_dir = '/work/ryan.martz/wofs_phi_data/experiments'
########################## Calc BSS ##########################

num_folds = 5
hazards = ['hail', 'wind', 'tornado']
radii = [39]
wofs_spinup_time = 25
forecast_length = 60
leads = [30, 60, 90, 120, 150, 180]
train_types = ['obs', 'obs_and_warnings']
model_type = 'wofs_psv2_no_torp'
use_avg_srs = True
for train_type in train_types:
    ver_type = train_type
    for hazard in hazards:
        for train_radius in radii:
            ver_radius = train_radius
            sr_skill_by_lead = np.zeros(len(leads))
            probs_skill_by_lead = np.zeros(len(leads))
            sr_skill_by_lead_fold = np.zeros((len(leads), num_folds))
            probs_skill_by_lead_fold = np.zeros((len(leads), num_folds))
            #start_i = get_bss_start_i(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = use_avg_srs)
            start_i = 0
            if not (start_i == 0):
                sr_skill_by_lead[0:start_i], probs_skill_by_lead[0:start_i], sr_skill_by_lead_fold[0:start_i,:], probs_skill_by_lead_fold[0:start_i,:] = get_done_skills(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = use_avg_srs)
            for i in range(start_i, len(leads)):
                lead = leads[i]
                m = model_stats(hazard, wofs_spinup_time, forecast_length, lead, train_radius, ver_radius, train_type, ver_type, model_type, num_folds, use_avg_srs)
                bss_sr, sr_skill_by_fold, bss_probs, probs_skill_by_fold = m.gen_brier_skill_scores()
                sr_skill_by_lead_fold[i,:] = sr_skill_by_fold
                probs_skill_by_lead_fold[i,:] = probs_skill_by_fold
                sr_skill_by_lead[i] = bss_sr
                probs_skill_by_lead[i] = bss_probs

            save_dir, overall_sr_file, overall_probs_file, sr_by_fold_file, probs_by_fold_file = get_bss_fname_dir(home_dir, hazard, train_radius, ver_radius, forecast_length, train_type, ver_type, model_type, use_avg_srs = use_avg_srs)

            utilities.save_data(save_dir, overall_sr_file, sr_skill_by_lead, 'npy')
            utilities.save_data(save_dir, overall_probs_file, probs_skill_by_lead, 'npy')
            utilities.save_data(save_dir, sr_by_fold_file, sr_skill_by_lead_fold, 'npy')
            utilities.save_data(save_dir, probs_by_fold_file, probs_skill_by_lead_fold, 'npy')

In [6]:
home_dir = '/work/ryan.martz/wofs_phi_data/experiments'
########################## Calc Reliability ##########################
'''Documentation on Reliability Summary Files (Content by Row):
1. Reliability Score from SR forecasts, Reliability Score from raw prob forecasts, Overall climo of forecasted event, 0s the rest of the row
2. The point forecasts to use for plotting (ex: 0.05, 0.15, 0.25, etc. for 0-0.1, 0.1-0.2, 0.2-0.3, etc.
3. The frequency of forecasts in each range for SR probs
4. The event climatology given such a forecast from the SRs
5. The frequency of forecasts in each range for raw probs
6. The event climatology given such a forecast from the raw probs'''

num_folds = 5
hazards = ['tornado', 'hail', 'wind']
radii = [39]
wofs_spinup_time = 25
forecast_length = 60
leads = [30, 60, 90, 120, 150, 180]
train_types = ['obs_and_warnings', 'obs']
ver_types = ['obs_and_warnings', 'obs']
model_type = 'wofs_psv2_no_torp'
use_avg_srs = True

for train_type in train_types:
    for ver_type in ver_types:
        for hazard in hazards:
            for train_radius in radii:
                ver_radius = train_radius
                for i in range(len(leads)):
                    lead = leads[i]
                    m = model_stats(hazard, wofs_spinup_time, forecast_length, lead, train_radius, ver_radius, train_type, ver_type, model_type, num_folds, use_avg_srs)
                    points = np.arange(0.05, 1, 0.1)
                    bins = np.arange(0, 1.01, 0.1)
                    fcst_frequencies_srs, fcst_frequencies_probs, rel_srs, rel_probs, rel_climos_srs, rel_climos_probs, ver_climo = m.calc_reliability(bins)
                    rel_summary_data = np.zeros((6, points.size))
                    rel_summary_data[0,0] = rel_srs
                    rel_summary_data[0,1] = rel_probs
                    rel_summary_data[0,2] = ver_climo
                    rel_summary_data[1,:] = points
                    rel_summary_data[2,:] = fcst_frequencies_srs
                    rel_summary_data[3,:] = rel_climos_srs
                    rel_summary_data[4,:] = fcst_frequencies_probs
                    rel_summary_data[5,:] = rel_climos_probs
                    save_dir = '%s/%s_trained/length_%s/%s' %(home_dir, train_type, forecast_length, hazard)
                    if train_type == 'warnings' and ver_type == 'warnings':
                        if use_avg_srs:
                            rel_fname = '%s_%s_trained_%s_verified_%s_%smin_reliability_data_avg_srs_%s-%s.npy' %(model_type, train_type, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            rel_fname = '%s_%s_trained_%s_verified_%s_%smin_reliability_data_%s-%s.npy' %(model_type, train_type, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                    elif train_type == 'warnings':
                        if use_avg_srs:
                            rel_fname = '%s_%s_trained_%s_%skm_verified_%s_%smin_reliability_data_avg_srs_%s-%s.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            rel_fname = '%s_%s_trained_%s_%skm_verified_%s_%smin_reliability_data_%s-%s.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                    elif ver_type == 'warnings':
                        if use_avg_srs:
                            rel_fname = '%s_%s_%skm_trained_%s_verified_%s_%smin_reliability_data_avg_srs_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            rel_fname = '%s_%s_%skm_trained_%s_verified_%s_%smin_reliability_data_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                    else:
                        if use_avg_srs:
                            rel_fname = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_reliability_data_avg_srs_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            rel_fname = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_reliability_data_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                    utilities.save_data(save_dir, rel_fname, rel_summary_data, 'npy')
                    time.sleep(2)
                    

IndexError: index 128880000 is out of bounds for axis 0 with size 128880000

In [ ]:
home_dir = '/work/ryan.martz/wofs_phi_data/experiments'
########################## Calc Performance Diagram Stats ##########################
'''Performance Diagram Summary Data Files by Row:
1. Forecast Thresholds for SR data
2. POD using SR probs
3. SR using SR probs
4. Forecast Thresholds for raw probs
5. POD using raw probs
6. SR using raw probs'''

num_folds = 5
hazards = ['tornado', 'hail', 'wind']
radii = [39]
wofs_spinup_time = 25
forecast_length = 60
leads = [30, 60, 90, 120, 150, 180]
train_types = ['obs_and_warnings', 'obs']
ver_types = ['obs_and_warnings', 'obs']
model_type = 'wofs_psv2_no_torp'
use_avg_srs = True

for train_type in train_types:
    for ver_type in ver_types:
        for hazard in hazards:
            for train_radius in radii:
                ver_radius = train_radius
                for i in range(len(leads)):
                    lead = leads[i]
                    m = model_stats(hazard, wofs_spinup_time, forecast_length, lead, train_radius, ver_radius, train_type, ver_type, model_type, num_folds, use_avg_srs)
                    if hazard == 'tornado':
                        thresholds = np.array([0, 0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
                    else:
                        thresholds = np.arange(0, 1.01, 0.1)
                    thresholds_sr, pods_sr, srs_sr, thresholds_probs, pods_probs, srs_probs = m.calc_pod_sr(thresholds)
                    pd_summary_data = np.zeros((6, thresholds.size))
                    pd_summary_data[0,:] = thresholds_sr
                    pd_summary_data[1,:] = pods_sr
                    pd_summary_data[2,:] = srs_sr
                    pd_summary_data[3,:] = thresholds_probs
                    pd_summary_data[4,:] = pods_probs
                    pd_summary_data[5,:] = srs_probs
                    
                    save_dir = '%s/%s_trained/length_%s/%s' %(home_dir, train_type, forecast_length, hazard)
                    if train_type == 'warnings' and ver_type == 'warnings':
                        if use_avg_srs:
                            pd_fname = '%s_%s_trained_%s_verified_%s_%smin_performance_diagram_data_avg_srs_%s-%s.npy' %(model_type, train_type, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            pd_fname = '%s_%s_trained_%s_verified_%s_%smin_performance_diagram_data_%s-%s.npy' %(model_type, train_type, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                    elif train_type == 'warnings':
                        if use_avg_srs:
                            pd_fname = '%s_%s_trained_%s_%skm_verified_%s_%smin_performance_diagram_data_avg_srs_%s-%s.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            pd_fname = '%s_%s_trained_%s_%skm_verified_%s_%smin_performance_diagram_data_%s-%s.npy' %(model_type, train_type, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                    elif ver_type == 'warnings':
                        if use_avg_srs:
                            pd_fname = '%s_%s_%skm_trained_%s_verified_%s_%smin_performance_diagram_data_avg_srs_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            pd_fname = '%s_%s_%skm_trained_%s_verified_%s_%smin_performance_diagram_data_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, hazard, forecast_length, lead, lead+forecast_length)
                    else:
                        if use_avg_srs:
                            pd_fname = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_performance_diagram_data_avg_srs_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                        else:
                            pd_fname = '%s_%s_%skm_trained_%s_%skm_verified_%s_%smin_performance_diagram_data_%s-%s.npy' %(model_type, train_type, train_radius, ver_type, ver_radius, hazard, forecast_length, lead, lead+forecast_length)
                    
                    utilities.save_data(save_dir, pd_fname, pd_summary_data, 'npy')
                    
                    time.sleep(2)
                    

In [19]:
########################## Transfer Ideal Models ##########################

home_dir = '/work/ryan.martz/wofs_phi_data/experiments'
sr_paste_dir = '/work/eric.loken/wofs/2024_update/SFE2024/sr_csv/latest'
model_paste_dir = '/work/eric.loken/wofs/2024_update/SFE2024/rf_models/latest'
model_type = 'wofs_psv2_no_torp'
hazards = ['hail', 'wind', 'tornado']
train_radii = [39]
wofs_lag = 25
length = 60
leads = [30, 60, 90, 120, 150, 180]
train_types = ['obs_and_warnings', 'obs']
use_avg_srs = True
for train_type in train_types:
    ver_type = train_type
    validation_dir = '/work/ryan.martz/wofs_phi_data/%s_train/validation_fcsts/%s' %(train_type, model_type)
    train_dir = '/work/ryan.martz/wofs_phi_data/%s_train/models/%s' %(train_type, model_type)
    for haz in hazards:
        for train_r in train_radii:
            ver_r = train_r
            for j in range(len(leads)):
                lead = leads[j]
                
                sr_skills_dir, __, __, sr_skills_fname, __ = get_bss_fname_dir(home_dir, haz, train_r, ver_r, length, train_type, ver_type, model_type, use_avg_srs = use_avg_srs)
                sr_skill_by_lead_time_fold = np.load('%s/%s' %(sr_skills_dir, sr_skills_fname))
                
                for i in range(len(sr_skill_by_lead_time_fold[:,0])):
                    ideal_fold = np.where(sr_skill_by_lead_time_fold[i,:] == max(sr_skill_by_lead_time_fold[i,:]))[0][0]
                    
                    if use_avg_srs:
                        sr_dir = '%s/%s/wofslag_%s/length_%s' %(validation_dir, haz, wofs_lag, length)
                    else:
                        sr_dir = '%s/%s/wofslag_%s/length_%s/all_raw_probs_fold%s' %(validation_dir, haz, wofs_lag, length, ideal_fold)
                    
                    model_dir = '%s/%s/wofslag_%s/length_%s' %(train_dir, haz, wofs_lag, length)
                    if train_type == 'warnings':
                        if use_avg_srs:
                            sr_fname = '%s_%s_trained_avg_sr_map_%s_spinup%smin_length%smin_%s-%s.csv' %(model_type, train_type, haz, wofs_lag, length, lead, lead+length)
                        else:
                            sr_fname = '%s_%s_trained_sr_map_%s_spinup%smin_length%smin_%s-%s_fold%s.csv' %(model_type, train_type, haz, wofs_lag, length, lead, lead+length, ideal_fold)
                        
                        sr_paste_name = '%s_%s-%smin_%s_sr_map.csv' %(train_type, lead, lead+length, haz)
                        model_fname = '%s_%s_trained_wofsphi_%s_%smin_window%s-%s_fold%s.pkl' %(model_type, train_type, haz, length, lead, lead+length, ideal_fold)
                        model_paste_fname = '%s_trained_wofsphi_%s_%smin_window%s-%s.pkl' %(train_type, haz, length, lead, lead+length)
                    else:
                        if use_avg_srs:
                            sr_fname = '%s_%s_trained_avg_sr_map_%s_spinup%smin_length%smin_%s-%s_r%skm.csv' %(model_type, train_type, haz, wofs_lag, length, lead, lead+length, r)
                        else:
                            sr_fname = '%s_%s_trained_sr_map_%s_spinup%smin_length%smin_%s-%s_r%skm_fold%s.csv' %(model_type, train_type, haz, wofs_lag, length, lead, lead+length, r, ideal_fold)
                        sr_paste_name = '%s_%skm_%s-%smin_%s_sr_map.csv' %(train_type, r, lead, lead+length, haz)
                        model_fname = '%s_%s_trained_wofsphi_%s_%smin_window%s-%s_r%skm_fold%s.pkl' %(model_type, train_type, haz, length, lead, lead+length, r, ideal_fold)
                        model_paste_fname = '%s_trained_wofsphi_%s_%smin_window%s-%s_r%skm.pkl' %(train_type, haz, length, lead, lead+length, r)
                    
                    full_sr_fname = '%s/%s' %(sr_dir, sr_fname)
                    full_model_fname = '%s/%s' %(model_dir, model_fname)
                    
                    
                    copy(full_sr_fname, '%s/%s' %(sr_paste_dir, sr_paste_name))
                    copy(full_model_fname, '%s/%s' %(model_paste_dir, model_paste_fname))
                    

In [32]:
top_dir = '/work/ryan.martz/wofs_phi_data/experiments'
train_types = ['obs', 'obs_and_warnings']
hazards = ['hail', 'wind', 'tornado']
length = 60
#### rename bss files:
for train_type in train_types:
    bss_dir = '%s/%s_trained' %(top_dir, train_type)
    for hazard in hazards:
        haz_dir = '%s/length_%s/%s' %(bss_dir, length, hazard)
        for file in os.listdir(haz_dir):
            if '_bss_' in file:
                splits = file.split('_')
                if 'and_fold' in file:
                    r_split = splits[-10]
                else:
                    r_split = splits[-8]
                new_fname1 = file[0:file.find(r_split)] + file[file.find(r_split)+len(r_split)+1:]
                new_fname2 = new_fname1[0:new_fname1.find('trained')] + r_split + '_' + new_fname1[new_fname1.find('trained'):]
                new_fname3 = new_fname2[0:new_fname2.find('verified')] + r_split + '_' + new_fname2[new_fname2.find('verified'):]
                os.rename('%s/%s' %(haz_dir, file), '%s/%s' %(haz_dir, new_fname3))
            else:
                continue

In [17]:
a = '30km'
a[0:-2]

'30'

In [28]:
file

'wofs_psv2_no_torp_obs_trained_obs_verified_hail_60min_7.5km_bss_from_raw_probs_by_lead_time_and_fold.npy'

In [27]:
file[0:file.find('trained')] + file[file.find('trained')+len('trained')+1:]

'wofs_psv2_no_torp_obs_obs_verified_hail_60min_7.5km_bss_from_raw_probs_by_lead_time_and_fold.npy'

In [25]:
file[0:file.find('trained')] + r_split + '_' + file[file.find('trained'):]

'wofs_psv2_no_torp_obs_7.5km_trained_obs_verified_hail_60min_7.5km_bss_from_raw_probs_by_lead_time_and_fold.npy'